<a href="https://colab.research.google.com/github/hyunkyungju/capstone-cnn/blob/main/capstone_resnet50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# torchvision 관련 라이브러리 import

from torchvision import utils
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

0. 데이터셋 로드

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install jsonlines
!pip install pdf2image

In [ ]:
#@title

import numpy as np

import torch
from torch import nn
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import torchvision.models as models
import tensorflow as tf
import keras


In [ ]:
#@title
from torch.utils.data import DataLoader

from torchvision import transforms
import torchvision
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import Dataset
import glob
from PIL import Image
import jsonlines
import os
from tqdm import tqdm

In [ ]:
#@title

class PaperDataSet(Dataset):
    def __init__(self, overall_image_path, transform=None):
        print("initialize data sets")
        self.transform = transform
        rating_dict = {}
        self.image_list = list()
        self.score_list = list()
        years = ["2021"]
        for year in years:
            cnt = 0
            year_image_path = overall_image_path+"iclr"+year+"/" #__test__/

            with jsonlines.open(f"{overall_image_path}/iclr{year}_metadata.jsonl") as read_file:
                for line in read_file.iter():
                    rating_dict[line['forum']] = line['rating']
            input_paths = os.listdir(year_image_path)
            for one_file_image_path in tqdm(input_paths, desc="make data set"):
                image_path = year_image_path + one_file_image_path + "/"
                before_add_size = len(self.image_list)
                self.image_list.extend(glob.glob(image_path + "*.jpg")) # glob: 폴더 내의 파일 찾아줌
                rating = rating_dict[one_file_image_path]
                self.score_list.extend([rating] * (len(self.image_list)-before_add_size))
                cnt += len(self.image_list)-before_add_size
            print(f"{year}: {cnt}")

    def __len__(self):
        return len(self.image_list)

    def __getitem__(self, idx):
        image_path = self.image_list[idx]
        label = self.score_list[idx]
        img = Image.open(image_path)
        if self.transform is not None:
            img = self.transform(img)

        return img, label

In [ ]:
image_path = "drive/Shareddrives/소종-논문/" 
dataset_file_name = 'iclr2021_dataset.pt'

In [ ]:
def make_save_data_set(image_path, dataset_file_name):
  print("start to make data set")
  transform = transforms.Compose([
      transforms.Resize((224, 224)),
      transforms.ToTensor(),
  ])

  dataset = PaperDataSet(image_path, transform=transform)
  print(f"data set length: {dataset.__len__()}")

  torch.save(dataset, dataset_file_name)
  print("save data sets")

In [ ]:
make_save_data_set(image_path, dataset_file_name)
# data_set_usage_ex(dataset_file_name)

In [ ]:
from torch.utils.data.dataset import random_split

dataset = torch.load(dataset_file_name)

train_size = int(0.8 * len(dataset))
print("train size:", train_size)

test_size = len(dataset) - train_size
print("test size:", test_size)
# validation 
train_dataset, test_dataset = random_split(dataset, [train_size,test_size])

train_dataloader = DataLoader(dataset=train_dataset, batch_size=50, shuffle=True)
test_dataloader = DataLoader(dataset=test_dataset, batch_size=32, shuffle=False)

02 training¶

In [ ]:
from torchvision import models
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') # 학습 환경 설정

resnet50 = models.resnet50(pretrained=False).to(device) # true 옵션으로 사전 학습된 모델을 로드

# transfer learning 사용 시 추가 
# if using_transfer_learning:
#   for param in resnet50.parameters():
#     param.requires_grad = False

model = nn.Sequential(
    resnet50,
    nn.Flatten(),
    nn.Linear(1000, 256),
    nn.ReLU(),
    nn.Linear(256, 32),
    nn.ReLU(),
    nn.Linear(32, 1)
).to(device)


In [ ]:
import torch
import torch.nn as nn
from torch import optim
lr = 0.0001
num_epochs = 10
optimizer = optim.Adam(model.parameters(), lr=lr)
loss_function = nn.MSELoss().to(device)

In [ ]:
params = {
    'num_epochs':num_epochs,
    'optimizer':optimizer,
    'loss_function':loss_function,
    'train_dataloader':train_dataloader,
    'test_dataloader': test_dataloader,
    'device':device
}

In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
import time
import datetime
from numpy import vstack

def train(model, params):
    total_start = time.time()
    loss_function=params["loss_function"]
    train_dataloader=params["train_dataloader"]
    test_dataloader=params["test_dataloader"]
    device=params["device"]

    print("start train")
    print("train size:", train_size)
    print("test size:", test_size)
    for epoch in range(0, num_epochs):
      epoch_start = time.time()
      trained_number = 0
      for i, data in enumerate(train_dataloader, 0):
        # train dataloader 로 불러온 데이터에서 이미지와 라벨을 분리
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.type(torch.LongTensor) 
        labels = labels.to(device)
        
        # 이전 batch에서 계산된 가중치를 초기화
        optimizer.zero_grad() 
        # forward + back propagation 연산
        outputs = model(inputs).squeeze()
        train_loss = loss_function(outputs.to(torch.float32), labels.to(torch.float32))
        train_loss.backward()
        optimizer.step()
        trained_number += labels.size(0)
        
        if i%100==0:
          print(f"epoch {epoch+1} {trained_number/train_size*100}% train finish")
      print(f"epoch {epoch+1} train finish") 
      # test accuracy 계산
      total = 0
      correct = 0
      loss = 0

      for i, data in enumerate(test_dataloader, 0):
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.type(torch.LongTensor) 
        labels = labels.to(device)
        

        # 결과값 연산
        outputs = model(inputs).squeeze()
        i_batch_size = labels.size(0)
        total += i_batch_size
        correct += (abs(outputs - labels)<0.5).sum().item() # 변경될 수 있음 
        test_loss = loss_function(outputs.to(torch.float32), labels.to(torch.float32)).item()
        loss += i_batch_size * test_loss
        if i%50==0:
          print(f"epoch {epoch+1} {total/test_size*100}% test finish")

      # 학습 결과 출력
      print('Epoch: %d/%d, Train loss: %.6f, Test loss: %.6f, Accuracy: %.2f' %(epoch+1, num_epochs, train_loss.item(), loss/total, 100*correct/total))

      epoch_elapsed_time = time.time() - epoch_start
      epoch_elapsed_time_list = str(datetime.timedelta(seconds=epoch_elapsed_time)).split(".")
      total_elapsed_time = time.time() - total_start
      total_elapsed_time_list = str(datetime.timedelta(seconds=total_elapsed_time)).split(".")
      print(f"Epoch {epoch+1} Elapsed time is {epoch_elapsed_time_list[0]}")  
      print(f"Total Elapsed time is {total_elapsed_time_list[0]}")  

In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()
train(model, params)